Die Bewegungsgleichung für 3 oder mehr Körper lässt sich im Allgemeinen nicht analytisch lösen. Numerisch gibt es verschieden anstäze:

#### Trivial Euler Verfahren:

Man wählt einen kleinen Zeitschritt $dt$, berechnet dann mit der Geschwindigkeit $v$ daraus die Koordinaten $r(t+dt)=r(t)+dt \cdot v(t)$ für den nächsten Schritt und aus den Koordinaten $r$ die Kraft $F(r)$ und damit die Beschleunigung $a=F/m$ auf den Körper und passt daraus die Geschwindigkeit $v(t+dt)=v(t)+a(t+dt) \cdot dt$ für den nächsten Zeitschritt an, um die Berechnung für diesen zu wiederholen. 
Dieses verfahren ist aber sehr instabil, denn selbst bei einer konstanten Beschleunigung ist die Geschwindigkeit mit der bei einem Zeitschritt gerechnet wird immer nur für den Anfangszeitpunkt des Zeitschrittes exakt und danach immer zu wenig in die beschleunigte Richtung. Für eine konstante Kraft kommt so in jedem Zeitschritt der gleiche Fehler hinzu und der summiert sich auf und wird immer größer. Beim Beispiel der Planetenbewegung ist die Kraft zwar nicht konstant, aber immer nach innen gerichtet, sodass der Planet bei dem Verfahren immer nicht weit genug nach innen beschleunigt wird, und immer weiter nach außen wegdriftet.

#### Besser Verlet Verfahren:

Vor dem ersten Schritt berechnet man die Kraft $F$ und die Beschleunigung $a=F/m$ auf den Körper entsprechend der Startkoordinaten, aber passt die Geschwindigkeit nur um die Beschleunigung für einen halben Zeitschritt an $v(t+½ \cdot dt) = v(t)+½ \cdot dt \cdot a(t)$. Dann werden in jedem Zeitschritt die Koordinaten $r(t+dt)=r(t)+v(t+½ \cdot dt) \cdot dt$ anhand der Geschwindigkeit zum Zeitpunkt in der Mitte des Zeitschrittes berechnet. Und die Geschwindigkeit $v(t+3/2 \cdot dt)=v(t+½ \cdot dt)+a(t+dt) \cdot dt$ wird anhand der Beschleunigung aus der Kraft zu den Koordinaten von ganzzahligen Zeitschritten berechnet, wobei das wieder die Beschleunigung zu dem Zeitpunkt ist, der genau in der Mitte zwischen den Zeitpunkten liegt, für die die Geschwindigkeiten berechnet wurden.
Also wäre die Rechnung schonmal für sich linear mit der Zeit ändernde Beschleunigungen und Geschwindigkeiten exakt und der Rechenaufwand ist kaum gestiegen
Das Verlet verfahren lässt sich auch über eine Taylorentwicklung zur 3. Ordnung von der Mitte in beide Richtungen herleiten, der Fehler liegt damit erst in 4. Ordnung. Die antisymmetrische Anordnung des 1. und 3. Terms lässt sich diese wegkürzen, sodass der Rechenaufwand niedrig bleibt.
Außerdem bleibt beim verlet-Verfahren das Volumen des Phasenraums erhalten.


## Solarsystem

Um ein Solarsystem nach dem Vorbild des Unseren zu simulieren haben wir in der Datei Solar.py die massen, 
Distanzen zur Sonne und Bahngeschwindigkeiten von den Planeten und unserem Mond definiert.

In [23]:
import Solar as sol
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt

## Einheiten

Beim verlet-Algorithmus werden für $n$ Iterationen mit dem Zeitschritt $h$ Position $\vec{r}$ und Geschwindigkeit $\vec{v}$ folgender Zeitpunkte ausgerechnet:
$$ \vec{r}(t_0 + h \cdot i) $$
$$ \vec{v}(t_0 - \frac{h}{2} + h \cdot i) $$
wobei
$$ i \in {1,...,n} $$
Die Werte für $\vec{r}(t)$ und $\vec{v}(t)$ werden dabei in folgendermaßen berechnet:
$$ \vec{r}(t) = \vec{r}(t-h) + h \cdot \vec{v}(t-\frac{h}{2})$$
$$ \vec{v}(t) = \vec{v}(t-h) + h \cdot \vec{a}(t-\frac{h}{2})$$
Dabei ist $\vec{a}(t)$ die Beschleunigung, die aus $\vec{r}(t)$ folgendermaßen berechnet wird:
$$ \vec{a} = \frac{\vec{F}}{m} $$
$$ \vec{F} = G \cdot m \cdot \sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
$$ \Rightarrow \vec{a} = G \cdot \sum_{j=1}^p\frac{m_j}{|\vec{r_j}-r|^2}$$
Dabei sind $m_j$ die Massen und $\vec{r_j}$ die Ortsvektoren der $p$ anderen Körper im System.
Um das ständige Multiplizieren mit der Gravitationskonstante $G$ wärend der Simulation zu vermeiden, können wir stattdessen einfach zu Beginn alle Massen jeweis mit ihr multiplizieren und diese Größe statt der Masse unter particle.mass speichern. Dann berechnet sich die Beschleunigung folgendermaßen:
$$ \vec{a} = \sum_{j=1}^p\frac{\lambda_j}{|\vec{r_j}-r|^2}$$
mit $$ \lambda_j = m_j \cdot G $$
Dabei sind $\lambda_j$ die mit der Gravitationskonstante multiplizierten massen particle.mass.
Die Massen werden im Code aber auch für die Berechnung der kinetischen und potentiellen Energie verwendet, also muss bei diesen Berechnungen die Masse durch $\frac{\lambda}{G}$ ersetzt werden:
$$ W_{pot}=G\frac{-m_1 \cdot m_2}{|\vec{r_1}-\vec{r_2}|} = \frac{1}{G} \cdot \frac{-\lambda_1 \cdot \lambda_2}{|\vec{r_1}-\vec{r_2}|}$$
$$ W_{kin}=\frac{1}{2} \cdot m \cdot v^2 = \frac{1}{G} \cdot \lambda \cdot v^2$$
Bei diesen Formeln sieht man dann, dass man die Gravitationskonstante einfach weglassen kann, und alle Energien werden mit dem gleichen Faktor skaliert, sodass alle Verhältnisse noch Stimmen.

In [ ]:
class Particle:
    def __init__(self, name, mass, coord, velocity, color):
        self.name = name
        self.mass = mass
        self.coord = np.array([coord["x"], coord["y"], coord["z"]])
        self.velocity = np.array([velocity["x"], velocity["y"], velocity["z"]])
        self.color = color

    def move(self, dt):
        # delta is the distance the particle is moved by in the timestep dt
        delta = self.velocity * dt
        self.coord = self.coord + delta

    def kineticEnergy(self):
        return self.mass / 2 * np.linalg.norm(self.velocity)**2

    def impulse(self):
        return self.mass * self.velocity

    def phi(self):
        radianHalf = np.cos(self.coord[0] / np.linalg.norm(np.array([self.coord[0], self.coord[1]])) )
        if self.coord[1] < 0:
            return 2 * np.pi - radianHalf
        return radianHalf 

    def potEnergy(self, otherParticle):
        diff = np.subtract(self.coord, otherParticle.coord)
        radius = np.linalg.norm(diff)
        return - self.mass * otherParticle.mass / radius

    def accelerate(self, other, dt):
        diff = np.subtract(other.coord, self.coord)
        radius = np.linalg.norm(diff)
        self.velocity = self.velocity + diff * other.mass * dt / (radius**3)
        other.velocity = other.velocity - diff * self.mass * dt / (radius**3)
        # return force that applys on self

## Particle Factory

Im folgenden definieren wir einige Sets an Himmelskörpern -> "Particles" für die folgenden Simulationen und
eine Methode um die Geschwindigkeiten von Particles in einer Liste so zu "shiften", dass der Schwerpunkt des Systems konstant ist. 

In [9]:
class ParticleFactory:

    def __init__(self, name):
        self.name = name
        self.particleList = self.getParticles(self)
        self.colors = [p.color for p in self.particleList]
        self.masses = [p.mass for p in self.particleList]
        maxsize = max(self.masses) ** (1/8)
        self.sizes = list(map(lambda x: x**(1/8) / maxsize * 69, self.masses))

    def getParticles(self):
        if self.name == "Solar":
            star = Particle("Sun", sol.mass_sun, {"x": 0, "y": 0, "z": 0}, {
                            "x": 0, "y": 0, "z": 0}, "yellow")
            # star = Particle("sun", 1000, {"x": 0, "y": 0, "z": 0 },{"x": -0.001, "y": 0, "z": 0 } )
            mercury = Particle("Mercury", sol.mass_mercury, {"x": 0, "y": sol.distance_mercury, "z": 0}, {
                "x": sol.velocity_mercury, "y": 0, "z": 0}, "#DDCC44")
            venus = Particle("Venus", sol.mass_venus, {"x": 0, "y": sol.distance_venus, "z": 0}, {
                "x": sol.velocity_venus, "y": 0, "z": 0}, "#884400")
            earth = Particle("Earth", sol.mass_earth, {"x": 0, "y": sol.distance_earth, "z": 0}, {
                "x": sol.velocity_earth, "y": 0, "z": 0}, "steelblue")
            mars = Particle("Mars", sol.mass_mars, {"x": 0, "y": sol.distance_mars, "z": 0}, {
                            "x": sol.velocity_mars, "y": 0, "z": 0}, "#EE1111")
            jupiter = Particle("Jupiter", sol.mass_jupiter, {"x": 0, "y": sol.distance_jupiter, "z": 0}, {
                "x": sol.velocity_jupiter, "y": 0, "z": 0}, "chocolate")
            saturn = Particle("Saturn", sol.mass_saturn, {"x": 0, "y": sol.distance_saturn, "z": 0}, {
                "x": sol.velocity_saturn, "y": 0, "z": 0}, "goldenrod")
            uranus = Particle("Uranus", sol.mass_uranus, {"x": 0, "y": sol.distance_uranus, "z": 0}, {
                "x": sol.velocity_uranus, "y": 0, "z": 0}, "lightseagreen")
            neptune = Particle("Neptune", sol.mass_neptune, {"x": 0, "y": sol.distance_neptune, "z": 0}, {
                "x": sol.velocity_neptune, "y": 0, "z": 0}, "cornflowerblue")
            moon = Particle("Moon", sol.mass_moon, {"x": sol.distance_moon, "y": sol.distance_earth, "z": 0}, {
                            "x": sol.velocity_earth, "y": sol.velocity_moon, "z": 0}, "#666666")
            return [star, mercury, venus, earth, mars, jupiter, saturn, uranus, moon]

        if self.name == "Tatoo":
            sun1 = Particle("Tatoo I", 50, {"x": 0, "y": 10, "z": 0}, {
                            "x": -1, "y": 0, "z": 0}, "peachpuff")
            sun2 = Particle(
                "Tatoo II", 50, {"x": 0, "y": -10, "z": 0}, {"x": 1, "y": 0, "z": 0}, "orange")
            planet = Particle("Planet", 1, {"x": 100, "y": 0, "z": 0}, {
                "x": 0, "y": 1, "z": 0}, "cadetblue")

            return [sun1, sun2]
            return [sun1, sun2, planet]

        if self.name == "Elipse":
            star = Particle("Star", 1000, {"x": 0, "y": 0, "z": 0}, {
                            "x": 0, "y": 0, "z": 0}, "orange")
            planet = Particle("Pluto", 1, {"x": 100, "y": 0, "z": 0}, {
                "x": 0, "y": 1.3, "z": 0}, "cadetblue")

            # return [sun1, sun2]
            return [star, planet]

        if self.name == "Moon System":
            star = Particle("Star", 1000, {"x": 0, "y": 0, "z": 0}, {
                            "x": 0, "y": 0, "z": 0}, "orange")
            planet = Particle("Planet", 10, {"x": 100, "y": 0, "z": 0}, {
                "x": 0, "y": 3, "z": 0}, "cadetblue")
            moon = Particle("Moon", 1, {"x": 100, "y": 4, "z": 0}, {
                            "x": 1, "y": 3, "z": 0}, "black")
            return [star, planet, moon]

        if self.name == "Lagrangepoints":
            star = Particle("Star", 1000, {"x": 0, "y": 0, "z": 0}, {
                            "x": 0, "y": 0, "z": 0}, "yellow")
            planet = Particle("Planet", 7, {"x": 100, "y": 0, "z": 0}, {
                "x": 0, "y": 3, "z": 0}, "green")
            # l1 = Particle("L1", 0.01, {"x": 100/(sqrt(50)+1)*sqrt(50), "y": 0, "z": 0}, {
            #                "x": 0, "y": 3/(sqrt(50)+1)*sqrt(50), "z": 0}, "black")
            # l2 = Particle("L2", 0.01, {"x": 100, "y": 0, "z": 0}, {
            #                "x": 0, "y": 3, "z": 0}, "black")
            l3 = Particle("L3", 0.01, {"x": -100, "y": 0, "z": 0}, {
                "x": 0, "y": -3, "z": 0}, "black")
            l4 = Particle("L4", 0.01, {"x": 100*np.cos(np.pi/3), "y": -100*np.sin(np.pi/3), "z": 0}, {
                "x": 3*np.sin(np.pi/3), "y": 3*np.cos(np.pi/3), "z": 0}, "black")
            l5 = Particle("L5", 0.01, {"x": 100*np.cos(np.pi/3), "y": 100*np.sin(np.pi/3), "z": 0}, {
                "x": -3*np.sin(np.pi/3), "y": 3*np.cos(np.pi/3), "z": 0}, "black")
            return [star, planet, l3, l4, l5]

        if self.name == "Phase Room":
            parts = [Particle("Star", 100000, {"x": 0, "y": 0, "z": 0}, {
                "x": 0, "y": 0, "z": 0}, "yellow")]

            for i in range(10):
                parts.append(Particle("Planet {}".format(i), 1, {"x": 0, "y": (
                    i+1)*500, "z": 0}, {"x": 10-i, "y": 0, "z": 0}, '#' + str(i)*6))
            return parts

    def removeOffsetSpeed(self):
        totalImpulse = np.array([float(0), float(0), float(0)])
        totalMass = 0
        for p in self.particleList:
            totalImpulse = np.add(totalImpulse, p.impulse())
            totalMass += p.mass
        totalVelocity = totalImpulse / totalMass
        for p in self.particleList:
            p.velocity = np.subtract(p.velocity, totalVelocity)
        return self.particleList


## Simulate

With Verlet

In [8]:
import matplotlib.pyplot as plt
import progressBar

class Simulate:
    """
    Class for Simulations

    @params
        particles   - Required  : array of Objects from Particle Class (Particle)
        h           - Required  : size of timestep (float)
        n           - Required  : number of timesteps (int)
    """

    def __init__(self, h: float, n: int ):
      
        self.h = h
        self.n = n

    def verlet(self, particles):
        
        """
        Calculates the movement of particles with Verlet algorithm
        Returns positions in multidimensional array [particle p][dimension 0-2][timestep i].
        Returns maximum and minimum of the coordinates too
       
        """

        mmin = np.amin(particles[0].coord)
        mmax = np.amax(particles[0].coord)
        t_axis = np.linspace(0, self.n*self.h, num=self.n)
        energy = np.zeros(self.n)
        p_axes = []
        p_impulses = []
        p_radius = []
        p_phi = []

        center =  np.array([float(0), float(0), float(0)])
        masssum = 0
        for p in particles:
            p_axes.append([np.zeros(self.n), np.zeros(self.n), np.zeros(self.n)])
            masssum += p.mass
            center += p.coord * p.mass 
            p_impulses.append(np.zeros(self.n))
            p_phi.append(np.zeros(self.n))
            p_radius.append(np.zeros(self.n))
        center *= (1/masssum)

        for pti in particles:
            for ptj in particles: 
                if ptj == pti:
                    break
                pti.accelerate(ptj, self.h/2)
        for i in range(self.n):
            for pt in particles:
                pt.move(self.h)
            for pti in particles:
                for ptj in particles: 
                    if pti == ptj:
                        break
                    energy[i] += pti.potEnergy(ptj)
                    #force = pti.gravityForce(ptj)
                    #pti.accelerate(force, self.h)
                    #ptj.accelerate(force, self.h)
                    pti.accelerate(ptj, self.h)
            for p, pt in enumerate(particles):
                energy[i] += pt.kineticEnergy()
                p_impulses[p][i] = np.linalg.norm(pt.impulse())
                p_radius[p][i] = np.linalg.norm(np.subtract(pt.coord, center) )
                p_phi[p][i] = pt.phi()
                p_axes[p][0][i] = pt.coord[0]
                p_axes[p][1][i] = pt.coord[1]
                p_axes[p][2][i] = pt.coord[2]
                mmin = min(mmin, np.amin(pt.coord))
                mmax = max(mmax, np.amax(pt.coord))
            if (i+1)/self.n*100%5 == 0: 
                progressBar.draw(i, self.n, "Verlet", "Complete", length=50)

        return (p_axes, p_impulses, p_radius, p_phi, energy, t_axis, mmin, mmax)

In [27]:
class Animations:
    """Animated scatter plot and charts using matplotlib.animations.FuncAnimation.

        @params
            particleFactory  - Required  : array of Objects from Particle Class (Particle)
            simulator        - Required  : method like Simulate.verlet(), that returns data arrays
    """

    def __init__(self, particleFactory: ParticleFactory, simulator):
        self.particleFactory = particleFactory
        self.p_axes, self.p_impulses, self.p_radius, self.p_phi, self.energy, self.t_axis, self.mmin, self.mmax = simulator(
            particleFactory.particleList)
        self.h = len(self.t_axis)
        self.n = self.t_axis[1]-self.t_axis[0]
        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        self.fig.set_figheight(10)
        self.fig.set_figwidth(10)
        plt.xlim([self.mmin, self.mmax])
        plt.ylim([self.mmin, self.mmax])
        # Then setup FuncAnimation.
        self.frames = 500
        self.fps = 25
        self.skip = int(self.n / self.frames)
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=int(1000/self.fps), frames=self.frames,
                                           init_func=self.setup_plot, blit=True)

        print("\n")
        print("Saving Video")
        self.ani.save('scatter.mp4', writer='ffmpeg', fps=self.fps,
                      dpi=100, metadata={'title': 'test'})

        # avoid plotting a spare static plot

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        xs = []
        ys = []
        for p in self.p_axes:
            xs.append(p[0][0])
            ys.append(p[1][0])
        self.scat = self.ax.scatter(
            xs, ys, s=self.particleFactory.sizes, c=self.particleFactory.colors)

        return self.scat,

    def update(self, i):
        """Update the scatter plot."""
        xy = []
        progressBar.draw(
            i, self.frames, prefix='Rendering', suffix='Complete', length=50)
        for p in self.p_axes:
            xy.append([p[0][i*self.skip], p[1][i*self.skip]])
        # Set x and y data...
        self.scat.set_offsets(xy)

        return self.scat,

    def pathPlot(self):

        fig, ax = plt.subplots()

        for p, pt in enumerate(self.particleFactory.particleList):
            ax.plot(self.p_axes[p][0],  self.p_axes[p][1], c=pt.color, label=pt.name)
        self.mmin -= (self.mmax - self.mmin) * 0.05
        self.mmax += (self.mmax - self.mmin) * 0.05
        fig.set_figwidth(10)
        fig.set_figheight(10)
        plt.xlim([self.mmin, self.mmax])
        plt.ylim([self.mmin, self.mmax])
        plt.legend()

        plt.title('{} - {} steps, dt={}'.format(self.particleFactory.name, self.n, self.h))
        plt.savefig('images/{} - {} steps, dt={}.png'.format(self.particleFactory.name, self.n, self.h))
        plt.close()

    def phaseSpace(self, is3d):
        if(is3d):
            fig3 = plt.figure()
            ax3 = fig3.add_subplot(projection='3d')

            for p, pt in enumerate(self.particleFactory.particleList):
                ax3.scatter(self.p_radius[p], self.p_phi[p], self.p_impulses[p], c=pt.color, s=4, label=pt.name)

            ax3.set_xlabel("radius")
            ax3.set_ylabel("impulse")
            #ax3.set_zlabel("phi")
            plt.title('{} - {} steps, dt={}'.format(self.particleFactory.name, self.n, self.h))
            plt.legend()
            plt.show()
            plt.savefig('images/{} - {} steps, dt={}_phase.png'.format(self.particleFactory.name, self.n, self.h))
            plt.close()
        else:
            fig3, ax3 = plt.subplots()
            for p, pt in enumerate(self.particleFactory.particles):
                ax3.plot(self.p_radius[p], self.p_impulses[p], c=pt.color, label=pt.name)

            plt.xlabel("radius")
            plt.ylabel("impulse")
            plt.title('{} - {} steps, dt={}'.format(self.particleFactory.name, self.n, self.h))
            plt.legend()
            plt.savefig('images/{} - {} steps, dt={}_phase.png'.format(self.particleFactory.name, self.n, self.h))
            plt.close()

        return (self.p_axes, self.mmin, self.mmax)